# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

from BB_agent_tool import BB_agent_tool, get_response, agent_query

import pdb

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [8]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [9]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [10]:
obb.user

UserSettings

id: 0664ab9d-bc3c-7dcc-8000-11faacd10505
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'intrinio_api_key': None, 'tradier_account_type': None, 'tradier_api_key': None, 'tradingeconomics_api_key': None, 'nasdaq_api_key': None, 'fred_api_key': SecretStr('**********'), 'polygon_api_key': SecretStr('**********'), 'fmp_api_key': SecretStr('**********'), 'tiingo_token': SecretStr('**********'), 'benzinga_api_key': None, 'biztoc_api_key': SecretStr('**********'), 'alpha_vantage_api_key': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [11]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [12]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()

In [13]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [14]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [15]:
symbol = "MRK"
company = "Merck"

In [16]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 0664ab9d-c4e1-7272-8000-d23ea51ada65
results: [{'symbol': 'MRK', 'one_day': 0.0024, 'wtd': None, 'one_week': 0.0087, 'mt...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [17]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=0.0024, wtd=None, one_week=0.0087, mtd=None, one_month=0.047599999999999996, qtd=None, three_month=0.026600000000000002, six_month=0.2944, ytd=0.2034, one_year=0.1432, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.0134, volatility_month=0.0149, price=131.19, volume=7722350.0, average_volume=8220000.000000001, relative_volume=0.94, analyst_recommendation=None, analyst_score=1.52)]

In [18]:
obj.dict()


{'id': '0664ab9d-c4e1-7272-8000-d23ea51ada65',
 'results': [{'symbol': 'MRK',
   'one_day': 0.0024,
   'wtd': None,
   'one_week': 0.0087,
   'mtd': None,
   'one_month': 0.047599999999999996,
   'qtd': None,
   'three_month': 0.026600000000000002,
   'six_month': 0.2944,
   'ytd': 0.2034,
   'one_year': 0.1432,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.0134,
   'volatility_month': 0.0149,
   'price': 131.19,
   'volume': 7722350.0,
   'average_volume': 8220000.000000001,
   'relative_volume': 0.94,
   'analyst_recommendation': None,
   'analyst_score': 1.52}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 231110750,
   'route': '/equity/price/performance',
   'timestamp': datetime.datetime(2024, 5, 19, 22, 

In [19]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 126.0,
   'bid_size': 1000,
   'ask': 133.0,
   'ask_size': 800,
   'last_price': 131.19,
   'open': 130.69,
   'high': 131.32,
   'low': 129.81,
   'volume': 8281595,
   'prev_close': 130.88,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 127.1046,
   'ma_200d': 114.8178,
   'volume_average': 8238545.0,
   'volume_average_10d': 6913350.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 302562416,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-19T22:47:56.398970'}}}

In [20]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-19,114.33,116.240,114.1000,115.49,7647080.0,115.4905,63052
2023-05-22,115.48,117.080,114.3300,114.49,5607422.0,115.0297,75291
2023-05-23,113.97,114.790,113.0500,113.27,7733023.0,113.5640,88433
2023-05-24,114.11,114.320,112.5500,113.60,4509534.0,113.6129,61615
2023-05-25,113.50,113.500,111.0900,112.30,7037608.0,112.0241,82967
...,...,...,...,...,...,...,...
2024-05-13,129.51,129.710,128.0200,129.29,6920607.0,128.9122,85432
2024-05-14,129.20,129.200,128.0269,128.66,6055317.0,128.5396,76232
2024-05-15,128.88,131.965,128.7200,131.73,7156188.0,130.9897,86852


In [21]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-19,418.62,420.7200,417.3500,420.17,642871,103793317.0,419.0232
2023-05-22,418.79,420.3900,417.3500,418.64,493531,60717899.0,419.0370
2023-05-23,414.09,418.7220,413.6800,417.08,624936,86383493.0,415.8938
2023-05-24,411.09,412.8200,409.8795,412.42,673413,89201678.0,411.2268
2023-05-25,414.65,416.1600,412.4101,414.74,672404,90961606.0,413.9115
...,...,...,...,...,...,...,...
2024-05-13,520.91,522.6700,519.7400,522.56,360847,36692999.0,521.0738
2024-05-14,523.30,523.8300,520.5600,521.11,417233,57521901.0,522.2996
2024-05-15,529.78,530.0800,525.1800,525.83,510546,59427333.0,528.0601


In [22]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [23]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,57.24,2024-05-17 20:00:01+00:00,57.02,57.380,56.7150,1669389,56.88,...,66.15,54.12,58.4552,60.03590,3012531,1.014350e+11,1772100000,4.65,12.31,2024-02-29 11:30:12+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,78.78,2024-05-17 20:00:02+00:00,78.30,79.220,77.7700,6429451,78.04,...,79.22,48.66,72.6396,63.42830,7072506,1.400141e+11,1777280000,2.39,32.96,2024-07-16 12:30:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,204.79,2024-05-17 20:00:08+00:00,203.81,205.045,202.8149,8991128,202.47,...,205.05,134.40,193.5998,167.13416,8851753,5.880893e+11,2871670000,16.57,12.36,2024-07-12 00:00:00+00:00
3,MS,Morgan Stanley,NYSE,100.22,2024-05-17 20:00:02+00:00,99.90,100.300,99.3000,4689785,99.58,...,100.98,69.42,92.4170,85.94165,7827641,1.628735e+11,1625160000,5.50,18.22,2024-07-16 12:30:00+00:00


In [24]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [25]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,id,amp_url,publisher
date,,,,,,,,,
2024-05-16 10:49:00+00:00,"The Zacks Analyst Blog Highlights Broadcom, Me...","Broadcom, Merck, Airbnb, ONEOK and PG&E are in...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2274496/the-z...,"PCG,MRK,OKE,AVGO,ABNB",Zacks Equity Research,rSveOM5z_sG_3ByZLLEbdQGcEV1fgmsxp81zzDegDYo,https://www.zacks.com/amp/stock/news/2274496/t...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-16 13:35:03+00:00,Merck (MRK) Crossed Above the 20-Day Moving Av...,When a stock breaks out above the 20-day simpl...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2274678/merck...,MRK,Zacks Equity Research,PvosUfrWiZe-WIwDjbx2DeQZtC6PTFYafAk51SyoR4Y,https://www.zacks.com/amp/stock/news/2274678/m...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-16 14:13:00+00:00,3 Drug Stocks to Watch on Raised 2024 Earnings...,"Here we discuss three drugmakers, Eli Lilly an...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2274825/3-dru...,"NVO,MRK,LLY",Kinjel Shah,xUp-v0S3tHM2CYzTNRJh_twmk1_qaHGKfGicQupe7HM,https://www.zacks.com/amp/stock/news/2274825/3...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-17 15:10:00+00:00,5 Stocks Powering the Dow ETF Year to Date,"The Dow Jones touched the milestone of 40,000 ...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2275517/5-sto...,"AMZN,GS,MRK,WMT,AXP,DIA",Sweta Killa,zg2zh4fQJkCT2JKqgPpqUcBsQlVM-tWk3ao9UBYB5zA,https://www.zacks.com/amp/stock/news/2275517/5...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-17 15:39:00+00:00,"Pharma Stock Roundup: BAYRY's Q1 Earnings, JNJ...",Bayer (BAYRY) releases Q1 results. J&J (JNJ) s...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2275562/pharm...,"JNJ,NVO,MRK,LLY,BAYRY",Kinjel Shah,stgIxTToXipTjGoYXkdRtTwSj9wP7mSF2euIUnFYIGk,https://www.zacks.com/amp/stock/news/2275562/p...,{'favicon_url': 'https://s3.polygon.io/public/...


# Prompt OpenAI 

In [26]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536     
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0

client = OpenAI()


In [27]:
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(client, messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The airspeed velocity of an unladen European swallow (Hirundo rustica) has been a topic of humor and curiosity, especially popularized by the film "Monty Python and the Holy Grail." According to some ornithological studies and estimations, the average cruising speed of an unladen European swallow is roughly 20 to 24 miles per hour (approximately 32 to 39 kilometers per hour).

If you were referring to an African swallow, it’s worth noting that there are many different species of swallows in Africa, and their speeds might vary slightly. However, the general flying speed for small birds like these would be in a similar range.

This humorous question highlights the kind of whimsical debates the film is known for, combining real-world science with comedic effect.


# Question Answering Agent

In [28]:
# make a custom tool 

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = tool(symbol="MSFT")
len(json.loads(tool_response)[0]['item1'])


143226

In [29]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(client, user_message, verbose=False)


get_10k_item1_from_symbol({'symbol': 'MSFT'}) -> [{"item1": "Note About Forward-Looking Statements\nThis report includes estimates, projections, stat...


### Summary of Item 1 from Microsoft's Latest 10-K Annual Report

#### General Overview
Microsoft is a technology company focused on empowering individuals and organizations globally to achieve more through its various technological offerings. Its mission is reflected in the development of platforms and tools powered by AI, aimed at enhancing business competitiveness, educational outcomes, public sector efficiency, and human ingenuity.

#### Business Segments
1. **Productivity and Business Processes:**
   - Products: Office 365, Microsoft Teams, LinkedIn, and Dynamics 365.
   - Focus: Reinventing productivity and business processes through advanced AI-backed tools that enhance creativity, collaboration, and innovation.

2. **Intelligent Cloud:**
   - Products: Azure, SQL Server, Windows Server, Visual Studio, and GitHub.
   - Focus: Building the intelligent cloud and intelligent edge platform, offering scalable cloud computing infrastructure and AI capabilities to solve business challenges.

3. **More Personal Computing:**
   - Products: Windows, Surface devices, Xbox, Microsoft Edge, Bing, and HoloLens.
   - Focus: Enhancing personal computing through leading-edge software and hardware, innovative device categories, and immersive gaming experiences.

#### Strategic Ambitions
1. **Reinvent Productivity and Business Processes:** Utilizes products like Office 365 and Microsoft Teams to increase security, productivity, and collaboration within workplaces.
2. **Build the Intelligent Cloud and Intelligent Edge Platform:** Empowers organizations with scalable AI solutions, cloud infrastructure, and emerging technologies like IoT.
3. **Create More Personal Computing:** Advances user interaction with technology through user-friendly interfaces, secure operating systems, and innovative devices like Surface and HoloLens.

#### Investment in AI
Microsoft's AI investments span across its product offerings. Key initiatives include:
   - Integration of AI into products like Microsoft Teams, Outlook, and Azure.
   - Development of generative AI capabilities to serve both consumer and commercial needs.
   - Commitment to ethical AI principles such as fairness, reliability, privacy, and inclusiveness.
   - The long-term partnership with OpenAI to advance AI research and deployment.

#### Commitment to Corporate Social Responsibility
1. **Sustainability:** Aims to be carbon negative, water positive, and zero waste by 2030. Progress includes reducing overall emissions and expanding renewable energy.
2. **Racial Equity:** Addresses racial injustice with initiatives targeting community strengthening, ecosystem engagement, and increased diversity within Microsoft.
3. **Digital Skills Training:** Focused on skilling jobseekers and professionals in digital and cybersecurity fields, driven by tools like LinkedIn, GitHub, and Microsoft Learn.

#### Human Capital
Microsoft values growth mindset and inclusivity, focusing on recruiting, developing, and retaining diverse talent. It offers a supportive work environment with development opportunities for all employees and emphasizes pay equity and well-being.

#### Operating Segments
1. **Productivity and Business Processes:** Office products and services aimed at increasing productivity, supported by AI.
2. **Intelligent Cloud:** Comprehensive cloud services and infrastructure supporting business transformation.
3. **More Personal Computing:** Focused on personal use through Windows, Surface, and Xbox products, enhancing customer computing experiences.

For a comprehensive understanding, please refer to Microsoft's full 10-K report.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [30]:
# TODO:

# move object to a file , move llm calls to a file

# make a streamlit app that maintains conversation

# use assistants API


In [31]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [75]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [76]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [77]:
fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            },""
        "limit": {
            "type": "integer",
            "description": "The number of results to return. Pick a small number from 1 to 10 and choose the best response."
            }
        },
    "example_parameter_values": [{
        "query": "Broadcom",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)



In [78]:
BB_agent_tool.agent_registry['get_equity_search_symbol'](query="Southwest Air")

'[]'

In [79]:
agent_query(client, "What is the stock symbol for Southwest airlines?", verbose=False)


get_equity_search_symbol({'query': 'Southwest Airlines', 'limit': 5}) -> []
get_equity_search_symbol({'query': 'Southwest', 'limit': 10}) -> [{"symbol": "CSWC", "name": "CAPITAL SOUTHWEST CORP COM", "dpm_name": "Wolverine Trading, LLC", "pos...


The stock symbol for Southwest Airlines is **LUV**.

In [37]:
agent_query(client, "What is the stock symbol for yakimee air?", verbose=False)


get_equity_search_symbol({'query': 'yakimee air', 'limit': 1}) -> []


It appears there's no direct match for "yakimee air." It might be a misspelling or not a publicly traded company. Could you please check the name or provide more details?

In [38]:
# agent is able to first get symbol, then get annual report for symbol 
agent_query(client, "What is item 1 from the 10k annual report for Microsfot?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nWhPL1NXYrMCtGZkYkJUJNhG', function=Function(arguments='{"query": "Microsfot", "limit": 1}', name='get_equity_search_symbol'), type='function'), ChatCompletionMessageToolCall(id='call_O3NX7YD9eIpBTDUh5cFS7WTK', function=Function(arguments='{"query": "microsoft", "limit": 1}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_nWhPL1NXYrMCtGZkYkJUJNhG', function=Function(arguments='{"query": "Microsfot", "limit": 1}', name='get_equity_search_symbol'), type='function'), ChatCompletionMessageToolCall(id='call_O3NX7YD9eIpBTDUh5cFS7WTK', function=Function(arguments='{"query": "microsoft", "limit": 1}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'query': 'Microsfot', 'limit': 1}) -> []
get_equity_search_symbol({'query': 'micro

Item 1 from the latest 10-K annual report for Microsoft (symbol: MSFT) provides comprehensive insight into various aspects of the company's operations, aspirations, and challenges. Here's a concise summary:

1. **Forward-Looking Statements:** Microsoft includes cautionary notes about forward-looking statements, indicating that their business is subject to risks and uncertainties which might cause actual results to differ materially from expectations.

2. **Business Overview:**
   - **Mission and Vision:** Microsoft's mission is to empower every individual and organization on the planet to achieve more. They aim to create opportunities and impact across global markets.
   - **AI and Cloud Services:** Microsoft emphasizes its commitment to AI, incorporating it throughout their product offerings including Microsoft Teams, Outlook, Bing, and Xbox. They are also focusing on AI capabilities within the Microsoft Cloud.
   - **Product Portfolio:** Microsoft develops an array of software, services, devices, and solutions, including their well-known operating systems, productivity applications, server and business solutions, and various devices like PCs and gaming consoles.
   
3. **Strategic Focus Areas:**
   - **Productivity and Business Processes:** With tools such as Office 365, Microsoft Teams, and Dynamics 365, Microsoft aims to reinvent productivity, enhance collaboration, and integrate their AI technologies to drive business processes.
   - **Intelligent Cloud and Edge Platform:** They invest heavily in cloud infrastructure, sustainability, and AI capabilities. Microsoft Azure plays a key role in their cloud strategy.
   - **Personal Computing:** Enhancing user interaction through innovative and secure devices, Microsoft continues developing products like Windows 11 and Surface.
   - **Gaming:** Through Xbox and acquiring companies like Activision Blizzard, Microsoft is focused on creating a connected gaming ecosystem.

4. **Human Capital and Corporate Culture:**
   - **Talent Development:** Microsoft strives to recruit, develop, and retain diverse and talented personnel. They emphasize a growth mindset culture and support various Employee Resource Groups to enhance workplace inclusion.
   - **Employee Wellbeing:** Offering extensive benefits, they focus on physical, emotional, and financial wellness, including support for hybrid working models.

5. **Corporate Social Responsibility:**
   - **Sustainability Initiatives:** Corporate commitments include becoming carbon negative, water positive, and zero waste by 2030.
   - **Racial Equity:** Focused on strengthening communities, engaging with ecosystems, and increasing representation within Microsoft.
   - **Digital Skills Training:** Initiatives to support job seekers worldwide with digital skills and, more recently, cybersecurity training.

6. **Risks and Challenges:** The report also acknowledges various strategic, competitive, and operational risks that Microsoft faces, including intense competition, cybersecurity threats, global market dynamics, and regulatory challenges.

This comprehensive focus on AI, cloud services, personal computing, gaming, and a commitment to sustainability and inclusivity is central to Microsoft's strategy moving forward.

In [40]:
agent_query(client, f"What is the last market quote for symbol {symbol}?", verbose=False)


I currently do not have the ability to fetch real-time market quotes. To obtain the latest market quote for Merck & Co., Inc. (ticker symbol: MRK), I recommend checking a financial news website or using a stock market app such as Yahoo Finance, Google Finance, or Bloomberg.

In [41]:
agent_query(client, f"What is the last market quote for Consolidated Agglomerators", verbose=False)


get_equity_search_symbol({'query': 'Consolidated Agglomerators', 'limit': 10}) -> []


It appears there's no match for "Consolidated Agglomerators" in the stock database. Could you please double-check the company name or provide more details?

In [42]:
agent_query(client, f"What is the current weather in Las Vegas", verbose=False)


It seems you are asking about weather information, which falls outside the scope of stock market-related queries that I can assist with. You can check the current weather in Las Vegas on various weather websites or apps like Weather.com, AccuWeather, or your local news station’s website. 

If you have any questions related to individual stocks or companies, feel free to ask!

In [43]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"Can you provide a basic company profile of symbol {symbol}", verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "name": "Merck & Co Inc", "cik": null, "cusip": null, "isin": null, "lei": null, ...


### Merck & Co Inc (Symbol: MRK)

- **Sector:** Healthcare
- **Industry Category:** Drug Manufacturers - General
- **Stock Exchange:** NYSE
- **Market Cap:** \\$332.28 Billion
- **Employees:** 72,000
- **Index:** DJIA, S&P 500
- **Institutional Ownership:** 79.11%
- **Beta:** 0.41

#### Description:
Merck & Co., Inc. is a health care company that provides health solutions through its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. It operates through the following segments:
- **Pharmaceutical:** Includes human health pharmaceutical and vaccine products.
- **Animal Health:** Discovers, develops, manufactures, and markets animal health products, such as pharmaceutical and vaccine products, for the prevention, treatment, and control of disease in livestock and companion animal species.
- **Other:** Consists of sales for the non-reportable segments of healthcare services.

The company was founded in 1891 and is headquartered in Rahway, NJ.

In [44]:
tool = BB_agent_tool(
    name="get_equity_shorts_short_interest",
    description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
    openapi_path='/api/v1/equity/shorts/short_interest',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


get_equity_shorts_short_interest({'symbol': 'MRK'}) -> [{"settlement_date": "2021-07-15", "symbol": "MRK", "issue_name": "Merck & Co., Inc.", "market_class...


Here are the latest statistics on short interest for Merck & Co., Inc. (ticker: MRK):

- **Settlement Date:** April 30, 2024
- **Current Short Position:** 18,754,018 shares
- **Previous Short Position:** 20,429,972 shares
- **Average Daily Volume:** 7,551,651 shares
- **Days to Cover:** 2.48 days
- **Change in Short Position:** -1,675,954 shares
- **Change Percentage:** -8.2%

If you need more detailed or specific information, feel free to ask!

In [45]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [46]:
agent_query(client, f"Provide historical split information for symbol AAPL", verbose=False)

get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


Here are the historical stock splits for Apple Inc. (AAPL):

1. **August 31, 2020**: 4-for-1 split
2. **June 9, 2014**: 7-for-1 split
3. **February 28, 2005**: 2-for-1 split
4. **June 21, 2000**: 2-for-1 split
5. **June 16, 1987**: 2-for-1 split

These splits demonstrate how Apple has adjusted its share price to keep it accessible to a broader group of investors over time.

In [47]:
data["paths"]["/api/v1/equity/fundamental/balance"]

{'get': {'tags': ['equity'],
  'summary': 'Balance',
  'description': 'Get the balance sheet for a given company.',
  'operationId': 'equity_fundamental_balance',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['fmp', 'intrinio', 'polygon', 'yfinance'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'symbol',
    'in': 'query',
    'required': True,
    'schema': {'type': 'string',
     'description': 'Symbol to get data for.',
     'title': 'Symbol'},
    'description': 'Symbol to get data for.'},
   {'name': 'period',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Time period of the data to return.',
     'default': 'annual',
     'title': 'Period'},
    'description': 'Time period of the data to return.'},
   {'name': 'limit',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'integer', 'minimum': 0}, {'type': 'null'}],
     'description':

In [48]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what are the latest total assets for symbol {symbol}", verbose=False)


get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The latest total assets for Merck & Co., Inc. (symbol: MRK) as of the fiscal year ending December 31, 2023, are \\$106.675 billion.

For more detailed financial information, you can refer to their latest [10-K filing on the SEC website](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [49]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [50]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent cash flow from operations for stock symbol {symbol}", verbose=False)


get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent cash flow from operations for Merck & Co., Inc. (symbol: MRK) as of the fiscal year ending December 31, 2023, was \\$13,006,000,000 USD. 

For further details, you can refer to their [10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [52]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent net income for symbol {symbol}", verbose=False)


get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent net income for Merck & Co., Inc. (symbol: MRK) as of the fiscal year ending on December 31, 2023, was \\$365 million. 

You can find more details in their latest filing [here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [53]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the most recent PE ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 332280000000.0, "pe_ratio": 146.04, "foward_pe": 13.2, "eps": 0.9, ...


The most recent PE ratio for Merck & Co., Inc. (symbol: MRK) is 146.04.

In [54]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent price to sales ratio for stock symbol {symbol}", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...


It appears there might have been a mix-up with the symbol "MRK." To clarify, are you asking about Merck & Co., Inc., the pharmaceutical company? If yes, let me proceed accordingly. 

In [55]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent revenue per share for stock symbol {symbol}", verbose=False)


get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol": "MRK", "revenue_per_share_ttm": 24.25424397947098, "net_income_per_share_ttm": 0.91038294...


The most recent revenue per share for Merck & Co., Inc. (symbol: MRK) is approximately \\$24.25.

In [56]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# 

In [57]:
# agent_query(client, f"what was the EPS for {company} in the quarter ended 2022-09-30?", verbose=False)


In [58]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the latest dividend data for stock symbol {symbol}?", verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for the stock symbol MRK (Merck & Co., Inc.) is as follows:

- **Ex-Dividend Date**: March 14, 2024
- **Dividend Amount**: \\$0.77 per share
- **Record Date**: March 15, 2024
- **Payment Date**: April 5, 2024
- **Declaration Date**: January 23, 2024

If you have any more questions or need further details, feel free to ask!

In [59]:
agent_query(client, f"what was the dividend data as of the end of 2022 for stock symbol {symbol}?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...
get_equity_search_symbol({'query': 'Merck', 'limit': 5}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


As of the end of 2022, Merck & Co Inc. (symbol: MRK) had the following dividend data:

- **Ex-Dividend Date:** December 14, 2022
- **Dividend Amount:** \\$0.73 per share
- **Record Date:** December 15, 2022
- **Payment Date:** January 9, 2023
- **Declaration Date:** November 29, 2022

In [60]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", verbose=False)


get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-18", "trailing_dividend_yield": 0.0249122807}, {"date": "2023-05-19", "trailing_d...


As of the end of 2023, the dividend yield for Merck & Co., Inc. (stock symbol: MRK) was approximately 2.81%.

In [61]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the performance for {company} from 1 year ago?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": 0.0024, "wtd": null, "one_week": 0.0087, "mtd": null, "one_month": 0.0...


The performance of Merck & Co., Inc. (symbol: MRK) over the past year shows a return of 14.32%. The current stock price is \\$131.19, with a volume of approximately 7.72 million shares being traded. The average volume is around 8.22 million shares.

In [62]:
# this might exceed token context making it unreliable
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [63]:
pd.DataFrame(json.loads(tool(symbol='MRK'))).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
450,MRK,FTXH,11684.0,0.075700,1.519621e+06
13,MRK,XLV,18454434.0,0.062123,2.400184e+09
78,MRK,QDVG.DE,1126933.0,0.061660,1.457012e+08
79,MRK,IUHE.AS,0.0,0.061095,1.451633e+08
80,MRK,IUHC.L,0.0,0.061095,1.451633e+08
...,...,...,...,...,...
682,MRK,GPAL,640.0,NaN,6.631040e+04
693,MRK,GDEF,397.0,NaN,5.170131e+04
703,MRK,EUNU.DE,50000.0,NaN,3.868500e+04
704,MRK,BBAG,48000.0,NaN,3.790191e+04


In [64]:
agent_query(client, f"which ETF has the highest weight in {company} ?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79663326.0, "weight": 0.0066, "market_val...


The ETF with the highest weight in Merck & Co., Inc. (ticker: MRK) is the Invesco S&P 500 Equal Weight Health Care ETF (symbol: **RYH**) with a weight of approximately **6.02%**.

In [70]:
# make a custom tool 

def get_largest_from_json(json_str, field_name):
    d = json.loads(json_str)
    df = pd.DataFrame(d).sort_values(field_name, ascending=False)
    return [{'largest': df.iloc[0].to_json()}]
    

tool = BB_agent_tool(
    name="get_largest_from_json",
    description="Given a json string json_str representing a list of objects, and a field name field_name, return the object with the largest value for field_name.",
    openapi_path=None,
    callable=get_largest_from_json,
    parameters={
        "json_str": {
            "type": "string",
            "description": "a JSON string representing a list of objects"
            },
        "field_name": {
            "type": "string",
            "description": "the field to search for the largest value"
            },
        },
    example_parameter_values=[{
        "json_str": '[{"name": "low", "value": 0}, {"name": "high", "value": 1}]',
        "field_name": 'value'
    }],
)

tool_response = tool(json_str='[{"name": "low", "value": 0}, {"name": "high", "value": 1}]',
                    field_name='value')
tool_response
# len(json.loads(tool_response)[0]['item1'])


'[{"largest": "{\\"name\\":\\"high\\",\\"value\\":1}"}]'

In [74]:
# write a function to return top 10 etfs with highest weight in company
agent_query(client, f"which ETF has the highest weight in {company} ?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 10}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79663326.0, "weight": 0.0066, "market_val...
get_largest_from_json({'json_str': '[{"equity_symbol": "MRK", "etf_symbol": "VGHAX", "shares": 20479587.0, "weight": 0.0571, "market_value": 2702281504.65}, {"equity_symbol": "MRK", "etf_symbol": "VGHCX", "shares": 20479587.0, "weight": 0.0571, "market_value": 2702281504.65}, {"equity_symbol": "MRK", "etf_symbol": "VHT", "shares": 8632795.0, "weight": 0.0537, "market_value": 1139097300.25}, {"equity_symbol": "MRK", "etf_symbol": "FHLC", "shares": 1275477.0, "weight": 0.05393190020000001, "market_value": 166934429.76}, {"equity_symbol": "MRK", "etf_symbol": "IYH", "shares": 1544493.0, "weight": 0.0602233, "market_value": 200089068.15}, {"equity_symbol": "MRK", "etf_symbol"

The ETF with the highest weight in Merck (MRK) is **XLV (Health Care Select Sector SPDR Fund)**, with a weight of **0.0621**.

In [73]:
[(k, t.description) for k, t in BB_agent_tool.agent_registry.items()]

[('get_10k_item1_from_symbol',
  "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing."),
 ('get_equity_search_symbol',
  'Given a search string, get the stock symbol of the top company whose name best matches the search string.'),
 ('get_company_profile_json',
  'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'),
 ('get_equity_shorts_short_interest',
  'Given a stock symbol, get data on short volume and days to cover in JSON format.'),
 ('get_equity_fundamental_historical_splits',
  "Given a stock symbol, get the company's historical stock splits in JSON format."),
 ('get_balance_sheet_json',
  'Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.'),
 ('get_cash_flow_json',
  'Given a stock symbol, get the latest cash flow statement data for the company in JSON format.'),
 ('get_income_statement_json',
  'Give

In [72]:
del BB_agent_tool.agent_registry['get_largest from json']

In [ ]:
# for k, t in BB_agent_tool.agent_registry.items():
#     print(t.example_code)

In [68]:
len(" ".join([(t.example_code) for k, t in BB_agent_tool.agent_registry.items()]))

23866